# 멋진 작사가 만들기 프로젝트

### 사용할 모듈 불러오기

In [1]:
import glob
import os, re
import numpy as np
import tensorflow as tf

---
### 데이터 읽어오기

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

# txt 파일을 모두 읽어서 raw_corpus 에 담아준다.
raw_corpus = []
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


---
### 데이터 정제하기

In [3]:
# 입력된 문장을 정제하는 함수!
# 1. 소문자로 바꾸고, 양쪽 공백을 지워준다
# 2. 특수문자 양쪽에 공백을 넣는다
# 3. 여러개의 공백은 하나의 공백으로 변경
# 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 변경
# 5. 다시 양쪽 공백을 지운다
# 6. 시작에 <start>, 끝에 <end>를 추가

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    
    return sentence

In [4]:
# 정제된 문장을 모을 리스트 
corpus = []

for sentence in raw_corpus:
    # 원하지 않는 문장은 건너뛰기!
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue

    # 정제된 문장을 담아준다
    preprocessed_sentence = preprocess_sentence(sentence)
    
    # 토큰 개수가 15개 이상인 문장을 걸러주기 위해서
    # count 변수를 만들고, 정제된 문장에 있는 띄어쓰기 개수를 센다.
    count = 0
    for i in preprocessed_sentence:
        if i == ' ':
            count +=1
    
    # 띄어쓰기 개수가 14개를 넘어가면 토큰이 15개 이상이라는 것!
    # 원하지 않는 문장은 건너뛰기!
    if count > 14: continue
    
    # 건너뛰지 않은 문장들만 corpus에 담긴다.
    corpus.append(preprocessed_sentence)
        
# 정제된 결과의 예시 확인~~
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용!
# 정제된 문장이 담긴 corpus를 받아 토큰화를 하는 함수
def tokenize(corpus):
    # 15000단어를 기억할 수 있는 tokenizer
    # 문장은 이미 정제됐으니 filter는 필요 없음
    # 15000단어에 포함되지 못한 단어는 '<unk>'로 변경
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    # tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰준다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰준다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7ff54a2e30d0>


In [6]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장 생성
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6263    3    0    0    0]
[  50    4   95  303   62   53    9  946 6263    3    0    0    0    0]


---
### 평가 데이터셋 분리

In [7]:
# 사이킷런의 train_test_split 함수를 사용해
# 소스 문장과 타겟 문장을 훈련 데이터와 평가 데이터로 분리
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          random_state=15
                                                         )

In [8]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


In [9]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 15000개와, 여기 포함되지 않은 0:<pad>를 포함하여 15001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만든다
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

---
### 인공지능 만들고 학습시키기

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 500
hidden_size = 2000
lyricist = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [12]:
lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=10, validation_data = (enc_val, dec_val))

Epoch 1/10
487/487 [==============================] - 309s 563ms/step - loss: 3.4652 - val_loss: 3.0077
Epoch 2/10
487/487 [==============================] - 285s 585ms/step - loss: 2.8692 - val_loss: 2.7795
Epoch 3/10
487/487 [==============================] - 285s 585ms/step - loss: 2.6023 - val_loss: 2.6091
Epoch 4/10
487/487 [==============================] - 285s 586ms/step - loss: 2.3310 - val_loss: 2.4719
Epoch 5/10
487/487 [==============================] - 285s 586ms/step - loss: 2.0672 - val_loss: 2.3644
Epoch 6/10
487/487 [==============================] - 286s 587ms/step - loss: 1.8173 - val_loss: 2.2857
Epoch 7/10
487/487 [==============================] - 286s 587ms/step - loss: 1.5911 - val_loss: 2.2307
Epoch 8/10
487/487 [==============================] - 285s 585ms/step - loss: 1.3964 - val_loss: 2.1953
Epoch 9/10
487/487 [==============================] - 285s 586ms/step - loss: 1.2374 - val_loss: 2.1891
Epoch 10/10
487/487 [==============================] - 286s 586m

---
### 테스트~

In [13]:
# 문장 만들어주는 함수
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어를 하나씩 예측해 문장을 만든다
    # 1. 입력받은 문장의 텐서를 입력한다
    # 2. 예측된 값 중 가장 높은 확률인 word index를 뽑아낸다
    # 3. 2에서 예측된 word index를 문장 뒤에 붙이고,
    # 또 붙이고, 붙이고, 반복하다가~
    # 4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성 끝!
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환한다
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [15]:
generate_text(lyricist, tokenizer, init_sentence="<start> i", max_len=20)

'<start> i m gonna be good to you <end> '

In [19]:
generate_text(lyricist, tokenizer, init_sentence="<start> you are", max_len=20)

'<start> you are the only one <end> '

In [20]:
generate_text(lyricist, tokenizer, init_sentence="<start> is", max_len=20)

'<start> is it just a lie <end> '

---
## 회고
- 토큰 개수 제한을 맞추는게 조금 어려웠다. 토큰이란 개념 자체가 생소해서 그랬던걸지도?..   
  띄어쓰기로 토큰을 나눈다는 걸 알게되서, 정제된 문장을 corpus에 넣는 과정에서,   
  문장마다 반복문으로 띄어쓰기 개수를 세어줬고, 띄어쓰기 개수가 일정치를 넘어가는 문장을 걸러냈다.   
  어떻게 하는건지 처음엔 감을 못잡고있다가, 제일 먼저 시도해봤던 방법이 바로 성공해서 기분 좋았다!   
- 루브릭 평가지표 중 validation loss를 2.2 밑으로 내리는걸 만족시켜보려고,   
  embedding_size와 hidden_size 값을 여러번 고쳐가면서 학습시켰는데,   
  한번 할때마다 4~50분이 걸렸다.
  학습시키는 시간이 길다보니까 기다리는게 고역이었다......   
  그래도 결국은 2.2 밑으로 내리는데 성공했다!!   
